### Creating a class for the project

In [ ]:
class mnist_nn:
  def __init__(self):
    ''' The init function would read the dataset from sklearn datasets and print its shape'''
    from keras.datasets import mnist
    (self.x_train,self.y_train),(self.x_test,self.y_test)=mnist.load_data()
    print('Size of Train data : ',self.x_train.shape)
    print('Size of Test data : ',self.x_test.shape)
    self.combine_x=[self.x_train,self.x_test]
    self.combine_y=[self.y_train,self.y_test]
  

  def ohe_outputs(self):
    ''' The function would one hot encode the outputs of the mnist dataset '''
    import numpy as np
    self.combine_ohe=[]
    for data_y in self.combine_y:
      result=np.zeros((data_y.shape[0],10))
      for i in range(data_y.shape[0]):
        result[i][data_y[i]]=1
      self.combine_ohe.append(result)

  def image_normalisation(self):
    ''' Dividing all pixel images by 255 to normalise it '''
    self.combine_x[0]=self.combine_x[0]/255
    self.combine_x[1]=self.combine_x[1]/255


  def create_best_model(self):
    


In [ ]:
mnist=mnist_nn()

Size of Train data :  (60000, 28, 28)
Size of Test data :  (10000, 28, 28)


In [ ]:
# Operations on image
mnist.ohe_outputs() # One Hot Encoding the Y values
mnist.image_normalisation() # Normalising the pixels in image